## 1. Set-up

In [1]:
!pip install -U simpletransformers sentence-transformers transformers -q
!pip install emoji unidecode pattern -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 25.5 MB/s eta 0:00:0

Copy datasets from Google Drive To Local VM to Avoid Disconnection

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!rm -r /content/data
!cp -r /content/drive/MyDrive/nlp/zindi/data /content

rm: cannot remove '/content/data': No such file or directory


Load Python Libraries

In [4]:
# data manipulation
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import *
import sklearn

# natural language processing - simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

# natural language processing -sentence transformers
from sentence_transformers import SentenceTransformer, util

# natrual language processing - transformers
import transformers
from transformers import AutoTokenizer, AutoModel, AdamW, AutoConfig, get_linear_schedule_with_warmup

# natural language processing - other libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.python.keras.callbacks import *

# text wragling
import re, string
from bs4 import BeautifulSoup
import emoji

# other libraries
import os, gc, tqdm, datetime, random

# customize printing
import warnings
warnings.simplefilter('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', None)

Environment Variables

In [5]:
seed = 1024

TRAIN_DATA_PATH = '/content/data/Train.csv'
TEST_DATA_PATH = '/content/data/Test.csv'
SAMPLE_SUB_PATH = '/content/data/SampleSubmission.csv'

TIMESTAMP = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
print(TIMESTAMP)

20230414-003459


In [6]:
# create a folder under local vm to save model results
os.makedirs('/content/models/', exist_ok=True)

Define Utility Functions
- [x] post-processing on predictions
- [x] save output to .csv

In [7]:
def postprocess_classification(preds):
  '''
    This function helps us go from a classifiaction
    problem to a regression one.
    The regression values range are in [-1, 1].
  '''
  final_preds = []
  for pred in preds:
    argmax = np.argmax(pred, axis=0)
    if argmax == 0: final_preds.append( -1*pred[0] )
    elif argmax == 1: final_preds.append( 0 )
    else: final_preds.append( pred[2] )
    
  return final_preds

In [8]:
def postprocess_regression(preds):
  '''
    This function clips the regression model output between range [-1, 1].
  '''
  preds_clipped = np.clip(preds, -1, 1)   
  return preds_clipped

In [9]:
def rmse(true, pred):
  '''This function calculates root mean squared error'''
  return np.sqrt(mean_squared_error(true, pred))

In [10]:
def prep_submission(df_pred, path_vm, path_gdrive, model_name, model_details, datetime):
  '''This function prepares submission file by
  1) save to .csv on local vm & designated folder on google drive
  2) download to local system 
  '''
  # save results to local vm and google drive
  filename = f'senti-reg_{model_name}_{model_details}_{datetime}.csv'
  df_pred.to_csv(os.path.join(path_vm, filename), index=False)
  df_pred.to_csv(os.path.join(path_gdrive,'/models', filename))

  # download to local system
  from google.colab import files
  files.download(os.path.join(path_vm, filename))

Import Datasets

In [11]:
train = pd.read_csv(TRAIN_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)

In [12]:
print('-'*10,'train','-'*10)
train.info()
train.sample(10)
print('-'*10,'test','-'*10)
test.info()
test.sample(10)
print('-'*10, 'sample submission', '-'*10)
sample_sub.info()
sample_sub.head(3)

---------- train ----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001 entries, 0 to 10000
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   10001 non-null  object 
 1   safe_text  10001 non-null  object 
 2   label      10000 non-null  float64
 3   agreement  9999 non-null   float64
dtypes: float64(2), object(2)
memory usage: 312.7+ KB


,tweet_id,safe_text,label,agreement
1319,5VVXFCWQ,Epidemiology tidbit: the Coolio vaccine successfully eradicated the public health menace in 2002.,0.0,0.666667
9899,H3LD67D2,".<user> it's so damn selfish, especially when some kids with weakened immune systems CAN'T be vaccinated on the proper schedule",1.0,0.666667
9687,IKDN50M5,Why you should vaccinate your kids.<url> via <user>,1.0,1.000000
177,9645UV58,Do not ask! What are all those bumps doing on me? I already had shingles and chicken pox and measles. <url>,0.0,1.000000
109,PNX57E59,"<user> let's use Ebola as a step to say #vaccineswork #flu is just as, if not more dangerous. It kills thousands. Also #MMR #polio",1.0,0.666667
515,4HPRNLHP,Are ppl aware that the majority of kids who got measles at Disneyland were UNVACCINATED? That means ur vaxx kids were not in danger so STFU!,0.0,0.666667
6924,WBZX647L,Access to Fitchburg school is limited amid measles scare <url> Most read by neighbors in #Fitchburg #MA,0.0,1.000000
3511,XRR2BZC0,The MMR ModSpace MX-5 team on the grid for the second Battery Tender race at Sebring! #Sebring #modspace #mazda #mx5cup #globalmx5cup #scca,0.0,1.000000
8526,ABTU0GHB,Meet me at #EscapeGentlemensClub Right Now‼️‼️ #JustTouchedDown #TheBreakupParty #MMR 😘 @ Escape… <url>,0.0,1.000000
8859,RTIP94D0,"My neck so sick got the measles, this fa my niggas trapping in the regals",0.0,1.000000


---------- test ----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet_id   5177 non-null   object
 1   safe_text  5176 non-null   object
dtypes: object(2)
memory usage: 81.0+ KB


,tweet_id,safe_text
1822,CHDBU27V,"SB550 [Update] In child protective services, further providing for definitions, for immunity from liability, for ... <url>"
3239,MDQJNR1E,<user> @ parents that think not vaccinating their children is a good thing
4770,X1ISQIUC,<user> #measles Is it a coincidence that we now have a measles problem after such a gigantic influx of illegal children?
2229,FH0D5KVN,Suspected case of measles in Wis. comes back negative: Portage County health officials said one of their suspected… <url>
4182,SV65RDNX,5 Babies Diagnosed With Measles At Chicago-Area Daycare Center - <url> #AntiVaxxer #antivax this is your doing. #medical.
5111,ZIKSRB23,"Vaccines, clean water, &amp; antibiotics most important to public health <user> #smilingitforward #ad"
3859,QLL3TNGU,<user> releases statement that #vaccines do not cause #autism! #MeaslesTruth #MeaslesOutbreak #vaccineswork <url>
2750,J36IZ34J,"""It's not just because I love my kids that I vaccinated them, it's because I love your kids as well."" <user> <url>"
1593,AXZVRFMS,"""Vaccinations cause autism"" ""my kid has a bad reaction"" bitch. No."
5013,YSGWVL81,“<user> Measles outbreak hits 13 states: <url> prime example of why you should vaccinate your children


---------- sample submission ----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  5177 non-null   object
 1   label     5177 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 81.0+ KB


,tweet_id,label
0,00BHHHP1,0
1,00UNMD0E,0
2,01AXPTJF,0


In [13]:
# check data imbalance
pd.DataFrame({'num_of_instances':train.label.value_counts(dropna=False), 
              '%':train.label.value_counts(normalize=True,dropna=False)})

,num_of_instances,%
0.000000,4908,0.490751
1.000000,4053,0.405259
-1.000000,1038,0.103790
NaN,1,0.000100
0.666667,1,0.000100


In [14]:
pd.DataFrame({'num_of_instances':train.agreement.value_counts(dropna=False),
              '%':train.agreement.value_counts(normalize=True,dropna=False)})

,num_of_instances,%
1.000000,5866,0.586541
0.666667,3894,0.389361
0.333333,239,0.023898
NaN,2,0.000200


In [15]:
# check missisng values
train.isnull().sum()
train[train.isnull().any(axis=1)]

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some parents could be so oblivious to reality as to not #vaccinate their child.,1,0.666667,NaN


In [16]:
test.isnull().sum()
test[test.isnull().any(axis=1)]

tweet_id     0
safe_text    1
dtype: int64

,tweet_id,safe_text
2024,Dr. JAMES SHANNON,NaN


In [17]:
# make sure all tweet_id in test file shows up in sample submission file
test[~test['tweet_id'].isin(sample_sub['tweet_id'])]
sample_sub[~sample_sub['tweet_id'].isin(test['tweet_id'])]

,tweet_id,safe_text
2024,Dr. JAMES SHANNON,NaN


,tweet_id,label
2452,H0VUUY2P,0


# 1. Data Cleaning
- [X] Remove NaN rows
- [X] Because some of the tweets were annotated multiple times by the same annotator, there can be duplicated rows. 
  - We first need to drop all rows, but one, with duplicated tweets where HandLabel is the same. 
  - After that we drop all duplicated tweets, since they all have different HandLabel and we do not know which one is correct. It would have been wrong if we dropped all the duplicates at once, without looking at the HandLabel, because we would threw away the highest quality data (the tweets which were labeled same multiple times).
- [x] All tweets are converted to lowercase
- [X] All links were removed since they do not contain any relevant information for this task and also '[video]' and '{link}' strings were removed because Twitter sometimes converts links to to these keywords.
- [X] A lot of tweets are usually retweets, that means that they contain 'RT @tweet_user' keywords, since 'RT @' is of no use it is replaces by '@'. ('@' is kept as indicator of tweet_user, because we will be removing them in the following steps). 
- [X] All usernames are removed. Usernames are words that start with '@'.
- [X] Dealing with hashtags: Hashtag symbol '#' is removed, but the words that follow that symbol are kept, since they usually contain a lot of useful information (they are usualy compressed representation of the tweet)



*Reference*:
- [Top 2 PyTorch + SimpleTransformers Solutions](https://github.com/rajat5ranjan/Zindi-Solutions/blob/master/To%20Vaccinate%20or%20Not%20to%20Vaccinate%20%23ZindiWeekendz/final_sub.ipynb)
- [Twitter-Sentiment-Analysis-RoBERTa](https://github.com/Data-Science-kosta/Twitter-Sentiment-Analysis-RoBERTa)
- [Twitter Sentiment Analysis with BERT + RoBERTa](https://www.kaggle.com/code/ludovicocuoghi/twitter-sentiment-analysis-with-bert-roberta)
- [Pre-Processing Tweets for Sentiment Analysis](https://medium.com/analytics-vidhya/pre-processing-tweets-for-sentiment-analysis-a74deda9993e)

In [18]:
train_clean = train.copy()
test_clean = test.copy()

### a. Missing values treatment

In [19]:
# training set
train_clean.iloc[4799,:] = ['MADEUPID', 'I cannot believe in this day and age some parents could be so oblivious to reality as to not #vaccinate their child.', 1.0, 0.666667]
train_clean.drop(index=[4798], inplace=True)
train_clean = train_clean[train_clean['label'].isin([-1.0, 0.0, 1.0])]
train_clean.reset_index(drop=True, inplace=True)

In [20]:
# test set
test_clean['safe_text'] = test_clean['safe_text'].fillna('Dr. JAMES SHANNON')
test_clean['tweet_id'] = np.where(test_clean['tweet_id']=='Dr. JAMES SHANNON', 'H0VUUY2P', test_clean['tweet_id'])

In [21]:
# audit - no missing values in both training and test sets
train_clean.isnull().sum()
test_clean.isnull().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

tweet_id     0
safe_text    0
dtype: int64

### b. Tweets Text Data Cleaning

In [22]:
def preprocess_tweets(df, params):
  clean_df = df.copy()

  #----------
  # define helper functions
  #----------
  # [Y] recommend to include
  def remove_user_mentions(clean_df):
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: re.compile('rt @').sub('@', x).strip())      # substitute 'RT @' with '@'
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: re.sub('\@[ a-zA-Z0-9]*', '', x))            # Remove usernames. The usernames are any word that starts with @.
    return clean_df

  def remove_extra_white_space(clean_df):
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.strip('.').strip())
    return clean_df

 # [tentative] 
  def remove_tweet_special_chars(clean_df):
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace('#', ' '))
    #clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace('\r', ' '))
    #clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace('\n', ' ')) # BERT may USE \r \n as sentence delimiters
    #clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace('&amp;', ' '))
    #clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace('_', ' '))
    #clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace(':', ' '))
    return clean_df

  # [NO] recommend not include
  def remove_numbers(clean_df):
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x:re.sub(r'\d+', '', x).strip())
    return clean_df

  def remove_punctuation(clean_df):
    pattern = r'[^\w\s]' 
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: re.sub(pattern,'', x))
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
    return clean_df

  def replace_weird_chars(clean_df):
    from unidecode import unidecode
    clean_df['safe_text'] = clean_df['safe_text'].apply(unidecode)
    #clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: re.sub(r'[^\x00-\x7f]',r'', x)) # [not working, maybe related to emoji] remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    return clean_df
  
  def url_to_words(raw_text):
    raw_text=str(raw_text).strip()
    soup = BeautifulSoup(raw_text, 'html.parser')
    raw_text = soup.text
    raw_text = emoji.demojize(raw_text)
    no_coms=re.sub(r'\.com','',raw_text)
    no_urls=re.sub('https?://www','',no_coms)
    no_urls1=re.sub('https?://','',no_urls)
    try:
        no_encoding=no_urls1.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        no_encoding = no_urls1
    # letters_only = re.sub("[^a-zA-Z0-9]", " ",no_encoding) 
    words = no_encoding.lower().split()                             
    # stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words] 
    return( " ".join( meaningful_words ))

  def process_eng_hashtag(input_text: str) -> str:
    return re.sub(r'#[a-z|A-Z]\S*',lambda m: ' '.join(re.findall('[A-Z][^A-Z]*|[a-z][^A-Z]*', m.group().lstrip('#'))), input_text)

  def lowercase(clean_df):
    clean_df['safe_text'] = clean_df['safe_text'].apply(str.lower)
    return clean_df

  def remove_patterns(df):
    #clean_df['safe_text'] = clean_df['safe_text'].str.replace('<user>|<url>|url|user|https?://\S+|www\.\S+|#\w+', '', regex=True)
    clean_df['safe_text'] = clean_df['safe_text'].str.replace('<user>|<url>', ' ', regex=True)
    return clean_df

 # [test]
  def remove_visual_patterns(clean_df):
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.lstrip('"'))
    clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: x.replace(':', ' '))
    return clean_df
  
  #----------
  # data preprocessing
  #----------
  # [yes] remove user mentions
  if params.get('remove_user_mentions', False): clean_df = remove_user_mentions(clean_df) 

  # [yes] remove extra space
  if params.get('remove_extra_space', False): clean_df = remove_extra_white_space(clean_df)

  # [no] convert url to words
  if params.get('url_to_words', False): clean_df['safe_text'] = clean_df['safe_text'].apply(url_to_words)

  # [no] split hashtag - has limitation on ALLCAPITALIZED words
  if params.get('process_eng_hashtag', False): clean_df['safe_text'] = clean_df['safe_text'].apply(lambda x: process_eng_hashtag(x))

  # [no] remove weird characters
  if params.get('replace_weird_chars', False): clean_df = replace_weird_chars(clean_df)

  # [no] remove numbers
  if params.get('remove_numbers', False): clean_df = remove_numbers(clean_df)

  # [no] remove punctuation
  if params.get('remove_punctuation', False): clean_df = remove_punctuation(clean_df)

  # [no for uncased model] convert to lowercase
  if params.get('lowercase', False): clean_df = lowercase(clean_df) 

  # [tentative] remove_tweet_special_chars
  if params.get('remove_tweet_special_chars', False): clean_df = remove_tweet_special_chars(clean_df) 

  # [good to keep] remove repeated patterns in tweets, such as <url>, <user>
  if params.get('remove_patterns', False): clean_df = remove_patterns(clean_df)

  # [test] remove visual patterns
  if params.get('remove_visual_patterns', False): clean_df = remove_visual_patterns(clean_df)

  return clean_df

In [23]:
params_preprocess = {
                     'remove_user_mentions':True,  # [YES]no.1 Boost performance, I'd like to keep. model does not analyze user name
                     'remove_extra_space':False,    # [YES]no.2 Boost performance, I'd like to keep
                     'remove_tweet_special_chars':False, # [TENTATIVE YES]no.3 boost performance, but cannot combine with remove_user_mentions
                     'lowercase':False,             # [NO] depends on models, tokenizer can take care of it
                     'remove_patterns':False,       # [NO] worse than do nothing, 0.50
                     'url_to_words':False,          # [NO] worse than do nothing
                     'split_hashtags':False,        # [NO] worse than do nothing. BERT can take care of hashtag based on internet discussion 
                     'replace_weird_chars':False,   # [NO] if apply unicode, not as good as do nothing; if remove non-acsii chars, even worse. [do not include]
                     'remove_numbers':False,        # [NO] worse than do nothing 
                     'remove_punctuation':False,    # [NO] worse than do nothing
                     'remove_visual_patterns':False  # [TEST]
                     }

In [24]:
train_clean = preprocess_tweets(train_clean, params_preprocess)
test_clean = preprocess_tweets(test_clean, params_preprocess)

In [25]:
# audit
from google.colab import data_table
data_table.enable_dataframe_formatter()

train_clean
test_clean

Output hidden; open in https://colab.research.google.com to view.

### c. Check Duplicates

In [26]:
# def remove_dups(df, dataset = 'train'):
#   if dataset != 'train':
#     print("Length of clean {} set is {}".format(dataset, len(df)))  
#     return df

#   # remove duplicates only on training set
#   def find_dups(df):
#     clean_df = df.copy()
#     dups_text = clean_df[clean_df.duplicated(subset=['safe_text'])]['safe_text']
#     dups = clean_df[clean_df['safe_text'].isin(dups_text)]
#     dups.sort_values(by='safe_text',inplace=True)
#     no_dups = clean_df[~clean_df['tweet_id'].isin(dups['tweet_id'])]
#     print('Found {} duplicates'.format(len(dups)))
#     return dups, no_dups

#   def get_highest_vote_for_dups(dups):
#     cnt = dups.groupby(['safe_text','label'],as_index=False).agg({'tweet_id':'nunique'}) 
#     highest_vote_dups = cnt[cnt.tweet_id == cnt.tweet_id.groupby(cnt['safe_text']).transform('max')]
#     #highest_vote_dups = cnt.loc[cnt.groupby(['safe_text'])['tweet_id'].idxmax()] 
#     highest_vote_dups = dups.merge(highest_vote_dups, on=['safe_text','label'], how='inner')
#     return highest_vote_dups

#   def remove_dups_1(dups):
#     '''
#     This function removes duplicates by take two steps approach:
#       step 1 We first need to drop all rows, but one, with duplicated tweets where label is the same.
#     '''
#     dedup_1 = dups.drop_duplicates(subset=['safe_text','label'], keep='first')
#     print(f'Dropped {len(dups) - len(dedup_1)} duplicate rows with same label')
#     return dedup_1

#   def remove_dups_2(dedup_1):
#     '''
#     This function removes duplicates by take two steps approach:
#       step 2 After that we drop all duplicated tweets, since they all have different label and we do not know which one is correct. 
#     '''
#     dedup_2 = dedup_1.drop_duplicates(subset=['safe_text'], keep=False)
#     print(f'Dropped {len(dedup_1) - len(dedup_2)} duplicate rows with different labels')
#     return dedup_2

#   dups, no_dups = find_dups(df)
#   highest_vote_dups = get_highest_vote_for_dups(dups)
#   dedup_1 = remove_dups_1(highest_vote_dups)
#   dedup_2 = remove_dups_2(dedup_1)
#   clean_df = pd.concat([no_dups, dedup_2])
  
#   # make sure no duplicates
#   assert clean_df['safe_text'].nunique() == clean_df.shape[0], 'duplicates still exist'
    
#   print("Length of clean {} set is {}".format(dataset, len(clean_df)))  
#   return clean_df

In [27]:
def remove_dups(df, dataset = 'train'):
  '''
  This function removes duplicates by take two steps approach:
    step 1 We first need to drop all rows, but one, with duplicated tweets where label is the same.
    step 2 After that we drop all duplicated tweets, since they all have different label and we do not know which one is correct. 
  '''
  clean_df = df.copy()
  n_rows = len(clean_df)

  if dataset == 'train':
    # step 1 - drop duplicated tweets with same label
    clean_df = clean_df.drop_duplicates(subset=['safe_text','label'], keep='first')
    print(f'Dropped {n_rows - len(clean_df)} duplicate rows with same label')

    # step 2 - there are only duplicates with different hand label, so drop both of them
    n_rows = len(clean_df)
    clean_df = clean_df.drop_duplicates(subset=['safe_text'], keep=False)
    print(f'Dropped {n_rows - len(clean_df)} duplicate rows with different labels')

    # make sure no duplicates
    assert clean_df['safe_text'].nunique() == clean_df.shape[0], 'duplicates still exist'
    
  print("Length of clean {} set is {}".format(dataset, len(clean_df)))  
  return clean_df

In [28]:
train_clean = remove_dups(train_clean, 'train')
test_clean = remove_dups(test_clean, 'test')

Dropped 309 duplicate rows with same label
Dropped 72 duplicate rows with different labels
Length of clean train set is 9619
Length of clean test set is 5177


In [29]:
# understand text statistics - text length
train_clean['safe_text'].apply(lambda x: len(x)).describe()
print('\n')
test_clean['safe_text'].apply(lambda x: len(x)).describe()

count    9619.000000
mean       99.463042
std        30.345802
min         3.000000
25%        78.000000
50%       107.000000
75%       122.000000
max       153.000000
Name: safe_text, dtype: float64

count    5177.000000
mean       98.704269
std        30.588390
min         6.000000
25%        77.000000
50%       105.000000
75%       122.000000
max       151.000000
Name: safe_text, dtype: float64

### d. Train/Validation/Test Split

In [30]:
def extract_text_and_y(df_train, df_test):

    # Training - This extraces the Train datasentece columns and pushes into variable text and one hot encodes the labels
    X_train_all = df_train[['tweet_id','safe_text']]
    y_train_all = df_train['label']

    #Test - This extracts the Test datasentence columns and pushes to a text
    X_test = df_test[['tweet_id','safe_text']]

    return X_train_all, y_train_all, X_test

In [31]:
X_train_all, y_train_all, X_test = extract_text_and_y(train_clean, test_clean)

# audit
print(len(X_train_all))
print(len(X_test))

9619
5177


In [32]:
# train validation split
def split_data(X_train_all, y_train_all, test_size, seed=seed):
  X_train, X_val, y_train, y_val = train_test_split(X_train_all, y_train_all, 
                                                    test_size=test_size,
                                                    stratify=y_train_all, 
                                                    random_state=seed)
  print('number of instances in training set:{}'.format(len(X_train)))
  print('number of instances in validation set:{}'.format(len(X_val)))
  return X_train, X_val, y_train, y_val

In [33]:
X_train, X_val, y_train, y_val = split_data(X_train_all, y_train_all, test_size=0.15)

number of instances in training set:8176
number of instances in validation set:1443


In [34]:
df_train = pd.concat([X_train, y_train], axis=1)
df_val = pd.concat([X_val, y_val], axis=1)
df_test = X_test

## 2. Feature Engineering
- [X] Extract Sentence Embeddings
- [X] Any other helpful features

#### a. Generate Sentence Embeddings Using Sentence Transformers

*Reference*:
- [Sentence Transformers Github Examples](https://github.com/UKPLab/sentence-transformers/tree/master/examples/training)
- [Sentence Transformer Document](https://www.sbert.net/docs/package_reference/SentenceTransformer.html)

In [35]:
models_sbert = ['sentence-transformers/paraphrase-mpnet-base-v2', 'all-mpnet-base-v2', 'all-distilroberta-v1', 'sentence-transformers/all-roberta-large-v1', 'all-MiniLM-L12-v2']

In [36]:
def get_sentence_embeddings(input_df, model_sbert=models_sbert):
  df = input_df.copy()
  text_data = df.safe_text.to_list()  
  model_nicknames = [mname.replace("sentence-transformers/", "").replace("-","_") for mname in models_sbert]
 
  for i, model_full_name in enumerate(models_sbert):

    # 1 - get embeddings from senetence transformers
    model = SentenceTransformer(model_full_name)
    embeddings = model.encode(text_data, show_progress_bar=True, convert_to_numpy=True)
    embeddings_lengh = len(embeddings[0])
    print('The length of {} embeddings is {}'.format(model_full_name, embeddings_lengh))

    # 2 - add sentence embeddings as features to pd.DataFrame
    col_embeddings = []
    emb_feats = [model_nicknames[i] + '_emb_' + str(j+1) for j in range(embeddings_lengh)]
    print(emb_feats[:5])
    df_emb_feats = pd.DataFrame.from_records(embeddings)
    df_emb_feats.columns = emb_feats
    df.reset_index(drop=True, inplace=True)
    df_emb_feats.reset_index(drop=True, inplace=True)
    df = pd.concat([df, df_emb_feats], axis=1)
  
  return df

In [37]:
df_train = get_sentence_embeddings(df_train)
df_val = get_sentence_embeddings(df_val)
df_test = get_sentence_embeddings(df_test)

Batches:   0%|          | 0/256 [00:00<?, ?it/s]

The length of sentence-transformers/paraphrase-mpnet-base-v2 embeddings is 768
['paraphrase_mpnet_base_v2_emb_1', 'paraphrase_mpnet_base_v2_emb_2', 'paraphrase_mpnet_base_v2_emb_3', 'paraphrase_mpnet_base_v2_emb_4', 'paraphrase_mpnet_base_v2_emb_5']


Batches:   0%|          | 0/256 [00:00<?, ?it/s]

The length of all-mpnet-base-v2 embeddings is 768
['all_mpnet_base_v2_emb_1', 'all_mpnet_base_v2_emb_2', 'all_mpnet_base_v2_emb_3', 'all_mpnet_base_v2_emb_4', 'all_mpnet_base_v2_emb_5']


Batches:   0%|          | 0/256 [00:00<?, ?it/s]

...

The length of all-distilroberta-v1 embeddings is 768
['all_distilroberta_v1_emb_1', 'all_distilroberta_v1_emb_2', 'all_distilroberta_v1_emb_3', 'all_distilroberta_v1_emb_4', 'all_distilroberta_v1_emb_5']


Batches:   0%|          | 0/162 [00:00<?, ?it/s]

The length of sentence-transformers/all-roberta-large-v1 embeddings is 1024
['all_roberta_large_v1_emb_1', 'all_roberta_large_v1_emb_2', 'all_roberta_large_v1_emb_3', 'all_roberta_large_v1_emb_4', 'all_roberta_large_v1_emb_5']


Batches:   0%|          | 0/162 [00:00<?, ?it/s]

The length of all-MiniLM-L12-v2 embeddings is 384
['all_MiniLM_L12_v2_emb_1', 'all_MiniLM_L12_v2_emb_2', 'all_MiniLM_L12_v2_emb_3', 'all_MiniLM_L12_v2_emb_4', 'all_MiniLM_L12_v2_emb_5']


#### b.Other Features

In [38]:
def other_features(df):
  df['tweet_length'] = df['safe_text'].apply(lambda x:len(x))
  df['num_words_no_numpunc'] = df['safe_text'].apply(lambda x: len(re.findall(r'\w+', x)))

  def count_punc(s):
    count = lambda l1,l2: sum([1 for x in l1 if x in l2])
    return count(s,set(string.punctuation))     

  df['num_puctuations'] = df['safe_text'].apply(count_punc)
  return df

In [39]:
df_train = other_features(df_train)
df_val = other_features(df_val)
df_test = other_features(df_test)

In [40]:
# audit
df_train.shape
df_val.shape
df_test.head()

(8176, 3718)

(1443, 3718)

tweet_id  \
0  00BHHHP1   
1  00UNMD0E   
2  01AXPTJF   
3  01HOEQJW   
4  01JUKMAO   

                                                                                                                                  safe_text  \
0                            <user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, FDA think just not worth the AE risk unfortunately.   
1                                                                Students starting school without whooping cough vaccinations <url> #scpick   
2                               I'm kinda over every ep of <user> being "ripped from the headlines." Measles? Let's get back to crime. #SVU   
3  How many innocent children die for lack of vaccination each year? Around 1.5 million. Too bad all their parents couldn't be here. #SB277   
4                  CDC eyeing bird flu vaccine for humans, though risk is low: Federal officials said Wednesday they're taking steps… <url>   

   paraphrase_mpnet_base_v2_emb_1  paraphrase_mpnet_base_v2_emb_2  \
0                        0.053033                       -0.017879   
1                       -0.054145                        0.196527   
2                        0.059703                        0.314626   
3                       -0.025989                        0.192078   
4                       -0.007208                        0.311534   

   paraphrase_mpnet_base_v2_emb_3  paraphrase_mpnet_base_v2_emb_4  \
0                        0.100040                       -0.027674   
1                        0.045782                       -0.102037   
2                        0.000975                       -0.016127   
3                       -0.024383                       -0.068721   
4                        0.087841                       -0.157677   

   paraphrase_mpnet_base_v2_emb_5  paraphrase_mpnet_base_v2_emb_6  \
0                        0.177951                        0.127811   
1                        0.171419                        0.057332   
2                        0.053624                        0.151787   
3                        0.115254                        0.104721   
4                        0.013071                        0.093465   

   paraphrase_mpnet_base_v2_emb_7  paraphrase_mpnet_base_v2_emb_8  \
0                        0.148674                       -0.107690   
1                        0.305280                       -0.141482   
2                        0.139473                       -0.098922   
3                        0.110931                       -0.083198   
4                       -0.169153                        0.029880   

   paraphrase_mpnet_base_v2_emb_9  paraphrase_mpnet_base_v2_emb_10  \
0                        0.229725                         0.166422   
1                        0.273961                        -0.066881   
2                       -0.024693                         0.058494   
3                        0.029611                         0.170036   
4                       -0.009279                        -0.001310   

   paraphrase_mpnet_base_v2_emb_11  paraphrase_mpnet_base_v2_emb_12  \
0                         0.084955                         0.014503   
1                         0.111474                        -0.140040   
2                         0.168402                         0.014904   
3                         0.254736                         0.051056   
4                         0.075249                        -0.106180   

   paraphrase_mpnet_base_v2_emb_13  paraphrase_mpnet_base_v2_emb_14  \
0                        -0.025003                         0.023420   
1                         0.016837                        -0.070771   
2                        -0.075609                        -0.054874   
3                         0.004764                        -0.158945   
4                         0.017140                        -0.045391   

   paraphrase_mpnet_base_v2_emb_15  paraphrase_mpnet_base_v2_emb_16  \
0                        -0.216561             

In [41]:
!rm -R /content/feat/
!rm -R /content/drive/MyDrive/nlp/zindi/feat
!mkdir /content/feat/

rm: cannot remove '/content/feat/': No such file or directory


In [42]:
df_train.to_csv('/content/feat/df_sent_emb_train.csv',index=False)
df_val.to_csv('/content/feat/df_sent_emb_val.csv', index=False)
df_test.to_csv('/content/feat/df_sent_emb_test.csv', index=False)

In [43]:
!cp -r /content/feat/ /content/drive/MyDrive/nlp/zindi

### 3. Shawllo ML

Baseline Model - Select Random Forest for Parallel Processing

*Reference*:
- [sklearn random forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

Prepare Input Data

In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as mse

In [45]:
cols = df_train.columns

In [46]:
cols_paraphrase_mpnet_base_v2 = [col for col in df_train.columns if 'paraphrase_mpnet_base_v2_' in col]
cols_all_mpnet_base_v2 = [col for col in df_train.columns if 'all_mpnet_base_v2_' in col]
cols_all_distilroberta_v1 = [col for col in df_train.columns if 'all_distilroberta_v1_' in col]
cols_all_roberta_large_v1 = [col for col in df_train.columns if 'all_roberta_large_v1_' in col]
cols_all_MiniLM_L12_v2 = [col for col in df_train.columns if 'all_MiniLM_L12_v2_' in col]
cols_other_feat = ['tweet_length','num_words_no_numpunc','num_puctuations']
print(len(cols_paraphrase_mpnet_base_v2));print(len(cols_all_mpnet_base_v2));print(len(cols_all_distilroberta_v1));print(len(cols_all_roberta_large_v1));print(len(cols_all_MiniLM_L12_v2))

768
768
768
1024
384


Attempt 1 - use all_mpnet_base_v2 sentence embeddings 

In [47]:
cols_exclude = ['tweet_id', 'safe_text', 'label'] + cols_paraphrase_mpnet_base_v2 + cols_all_distilroberta_v1 + cols_all_roberta_large_v1 + cols_all_MiniLM_L12_v2
cols_include = [col for col in cols if col not in cols_exclude]

In [48]:
X_train_final = df_train[cols_include]
y_train_final = df_train['label']
X_val_final = df_val[cols_include]
y_val_final = df_val['label']
X_test_final = df_test[cols_include]

print(X_train_final.columns)

Index(['all_mpnet_base_v2_emb_1', 'all_mpnet_base_v2_emb_2',
       'all_mpnet_base_v2_emb_3', 'all_mpnet_base_v2_emb_4',
       'all_mpnet_base_v2_emb_5', 'all_mpnet_base_v2_emb_6',
       'all_mpnet_base_v2_emb_7', 'all_mpnet_base_v2_emb_8',
       'all_mpnet_base_v2_emb_9', 'all_mpnet_base_v2_emb_10',
       ...
       'all_mpnet_base_v2_emb_762', 'all_mpnet_base_v2_emb_763',
       'all_mpnet_base_v2_emb_764', 'all_mpnet_base_v2_emb_765',
       'all_mpnet_base_v2_emb_766', 'all_mpnet_base_v2_emb_767',
       'all_mpnet_base_v2_emb_768', 'tweet_length', 'num_words_no_numpunc',
       'num_puctuations'],
      dtype='object', length=771)


In [49]:
reg = RandomForestRegressor(n_estimators = 700, 
                            max_depth = 10, 
                            max_features = "sqrt",
                            criterion="squared_error",
                            random_state = seed)

In [50]:
model = reg.fit(X_train_final,  y_train_final)

In [51]:
# traning set loss
y_pred_train = model.predict(X_train_final)
y_pred_train = np.clip(y_pred_train, -1, 1)
print(f"RMSE: {mse(y_train_final, y_pred_train)**0.5}")

RMSE: 0.4013041879479838


In [52]:
# Evaluation on Validation Set
y_pred_val = model.predict(X_val_final)
y_pred_val = np.clip(y_pred_val, -1, 1)

print(f"RMSE: {mse(y_val_final, y_pred_val)**0.5}")

RMSE: 0.5838306041823145


Attempt 2 - paraphrase_mpnet_base_v2

In [53]:
cols_exclude = ['tweet_id', 'safe_text', 'label'] + cols_all_mpnet_base_v2 + cols_all_distilroberta_v1 + cols_all_roberta_large_v1 + cols_all_MiniLM_L12_v2
cols_include = [col for col in cols if col not in cols_exclude]

In [54]:
X_train_final = df_train[cols_include]
y_train_final = df_train['label']
X_val_final = df_val[cols_include]
y_val_final = df_val['label']
X_test_final = df_test[cols_include]

print(X_train_final.columns)

Index(['paraphrase_mpnet_base_v2_emb_1', 'paraphrase_mpnet_base_v2_emb_2',
       'paraphrase_mpnet_base_v2_emb_3', 'paraphrase_mpnet_base_v2_emb_4',
       'paraphrase_mpnet_base_v2_emb_5', 'paraphrase_mpnet_base_v2_emb_6',
       'paraphrase_mpnet_base_v2_emb_7', 'paraphrase_mpnet_base_v2_emb_8',
       'paraphrase_mpnet_base_v2_emb_9', 'paraphrase_mpnet_base_v2_emb_10',
       ...
       'paraphrase_mpnet_base_v2_emb_762', 'paraphrase_mpnet_base_v2_emb_763',
       'paraphrase_mpnet_base_v2_emb_764', 'paraphrase_mpnet_base_v2_emb_765',
       'paraphrase_mpnet_base_v2_emb_766', 'paraphrase_mpnet_base_v2_emb_767',
       'paraphrase_mpnet_base_v2_emb_768', 'tweet_length',
       'num_words_no_numpunc', 'num_puctuations'],
      dtype='object', length=771)


In [55]:
reg = RandomForestRegressor(n_estimators = 700, 
                            max_depth = 10, 
                            max_features = "sqrt",
                            criterion="squared_error",
                            random_state = seed)

In [56]:
model = reg.fit(X_train_final,  y_train_final)

In [57]:
# traning set loss
y_pred_train = model.predict(X_train_final)
y_pred_train = np.clip(y_pred_train, -1, 1)
print(f"RMSE: {mse(y_train_final, y_pred_train)**0.5}")

RMSE: 0.39525541929191194


In [58]:
# Evaluation on Validation Set
y_pred_val = model.predict(X_val_final)
y_pred_val = np.clip(y_pred_val, -1, 1)

print(f"RMSE: {mse(y_val_final, y_pred_val)**0.5}")

RMSE: 0.5840727193818535


Attempt 3 - all_distilroberta_v1

In [59]:
cols_exclude = ['tweet_id', 'safe_text', 'label'] + cols_all_mpnet_base_v2 + cols_paraphrase_mpnet_base_v2 + cols_all_roberta_large_v1 + cols_all_MiniLM_L12_v2
cols_include = [col for col in cols if col not in cols_exclude]

In [60]:
X_train_final = df_train[cols_include]
y_train_final = df_train['label']
X_val_final = df_val[cols_include]
y_val_final = df_val['label']
X_test_final = df_test[cols_include]

print(X_train_final.columns)

Index(['all_distilroberta_v1_emb_1', 'all_distilroberta_v1_emb_2',
       'all_distilroberta_v1_emb_3', 'all_distilroberta_v1_emb_4',
       'all_distilroberta_v1_emb_5', 'all_distilroberta_v1_emb_6',
       'all_distilroberta_v1_emb_7', 'all_distilroberta_v1_emb_8',
       'all_distilroberta_v1_emb_9', 'all_distilroberta_v1_emb_10',
       ...
       'all_distilroberta_v1_emb_762', 'all_distilroberta_v1_emb_763',
       'all_distilroberta_v1_emb_764', 'all_distilroberta_v1_emb_765',
       'all_distilroberta_v1_emb_766', 'all_distilroberta_v1_emb_767',
       'all_distilroberta_v1_emb_768', 'tweet_length', 'num_words_no_numpunc',
       'num_puctuations'],
      dtype='object', length=771)


In [61]:
reg = RandomForestRegressor(n_estimators = 700, 
                            max_depth = 10, 
                            max_features = "sqrt",
                            criterion="squared_error",
                            random_state = seed)

In [62]:
model = reg.fit(X_train_final,  y_train_final)

In [63]:
# traning set loss
y_pred_train = model.predict(X_train_final)
y_pred_train = np.clip(y_pred_train, -1, 1)
print(f"RMSE: {mse(y_train_final, y_pred_train)**0.5}")

RMSE: 0.4066833894024782


In [64]:
# Evaluation on Validation Set
y_pred_val = model.predict(X_val_final)
y_pred_val = np.clip(y_pred_val, -1, 1)

print(f"RMSE: {mse(y_val_final, y_pred_val)**0.5}")

RMSE: 0.5900153948591156


Attempt 4 - all_roberta_large_v1

In [65]:
cols_exclude = ['tweet_id', 'safe_text', 'label'] + cols_all_mpnet_base_v2 + cols_paraphrase_mpnet_base_v2 + cols_all_distilroberta_v1 + cols_all_MiniLM_L12_v2
cols_include = [col for col in cols if col not in cols_exclude]

In [66]:
X_train_final = df_train[cols_include]
y_train_final = df_train['label']
X_val_final = df_val[cols_include]
y_val_final = df_val['label']
X_test_final = df_test[cols_include]

print(X_train_final.columns)

Index(['all_roberta_large_v1_emb_1', 'all_roberta_large_v1_emb_2',
       'all_roberta_large_v1_emb_3', 'all_roberta_large_v1_emb_4',
       'all_roberta_large_v1_emb_5', 'all_roberta_large_v1_emb_6',
       'all_roberta_large_v1_emb_7', 'all_roberta_large_v1_emb_8',
       'all_roberta_large_v1_emb_9', 'all_roberta_large_v1_emb_10',
       ...
       'all_roberta_large_v1_emb_1018', 'all_roberta_large_v1_emb_1019',
       'all_roberta_large_v1_emb_1020', 'all_roberta_large_v1_emb_1021',
       'all_roberta_large_v1_emb_1022', 'all_roberta_large_v1_emb_1023',
       'all_roberta_large_v1_emb_1024', 'tweet_length', 'num_words_no_numpunc',
       'num_puctuations'],
      dtype='object', length=1027)


In [67]:
reg = RandomForestRegressor(n_estimators = 700, 
                            max_depth = 10, 
                            max_features = "sqrt",
                            criterion="squared_error",
                            random_state = seed)

In [68]:
model = reg.fit(X_train_final,  y_train_final)

In [69]:
# traning set loss
y_pred_train = model.predict(X_train_final)
y_pred_train = np.clip(y_pred_train, -1, 1)
print(f"RMSE: {mse(y_train_final, y_pred_train)**0.5}")

RMSE: 0.3898131104034295


In [70]:
# Evaluation on Validation Set
y_pred_val = model.predict(X_val_final)
y_pred_val = np.clip(y_pred_val, -1, 1)

print(f"RMSE: {mse(y_val_final, y_pred_val)**0.5}")

RMSE: 0.5806698127008049


Attempt 5 - all_MiniLM_L12_v2

In [71]:
cols_exclude = ['tweet_id', 'safe_text', 'label'] + cols_all_mpnet_base_v2 + cols_paraphrase_mpnet_base_v2 + cols_all_distilroberta_v1 + cols_all_roberta_large_v1
cols_include = [col for col in cols if col not in cols_exclude]

In [72]:
X_train_final = df_train[cols_include]
y_train_final = df_train['label']
X_val_final = df_val[cols_include]
y_val_final = df_val['label']
X_test_final = df_test[cols_include]

print(X_train_final.columns)

Index(['all_MiniLM_L12_v2_emb_1', 'all_MiniLM_L12_v2_emb_2',
       'all_MiniLM_L12_v2_emb_3', 'all_MiniLM_L12_v2_emb_4',
       'all_MiniLM_L12_v2_emb_5', 'all_MiniLM_L12_v2_emb_6',
       'all_MiniLM_L12_v2_emb_7', 'all_MiniLM_L12_v2_emb_8',
       'all_MiniLM_L12_v2_emb_9', 'all_MiniLM_L12_v2_emb_10',
       ...
       'all_MiniLM_L12_v2_emb_378', 'all_MiniLM_L12_v2_emb_379',
       'all_MiniLM_L12_v2_emb_380', 'all_MiniLM_L12_v2_emb_381',
       'all_MiniLM_L12_v2_emb_382', 'all_MiniLM_L12_v2_emb_383',
       'all_MiniLM_L12_v2_emb_384', 'tweet_length', 'num_words_no_numpunc',
       'num_puctuations'],
      dtype='object', length=387)


In [73]:
reg = RandomForestRegressor(n_estimators = 700, 
                            max_depth = 10, 
                            max_features = "sqrt",
                            criterion="squared_error",
                            random_state = seed)

In [74]:
model = reg.fit(X_train_final,  y_train_final)

In [75]:
# traning set loss
y_pred_train = model.predict(X_train_final)
y_pred_train = np.clip(y_pred_train, -1, 1)
print(f"RMSE: {mse(y_train_final, y_pred_train)**0.5}")

RMSE: 0.4197994114792734


In [76]:
# Evaluation on Validation Set
y_pred_val = model.predict(X_val_final)
y_pred_val = np.clip(y_pred_val, -1, 1)

print(f"RMSE: {mse(y_val_final, y_pred_val)**0.5}")

RMSE: 0.5929265386230002


Attempt 6 - all embeddings at once

In [77]:
cols_exclude = ['tweet_id', 'safe_text', 'label'] #+ cols_all_mpnet_base_v2 + cols_paraphrase_mpnet_base_v2 + cols_all_distilroberta_v1 + cols_all_roberta_large_v1
cols_include = [col for col in cols if col not in cols_exclude]

In [78]:
X_train_final = df_train[cols_include]
y_train_final = df_train['label']
X_val_final = df_val[cols_include]
y_val_final = df_val['label']
X_test_final = df_test[cols_include]

print(X_train_final.columns)

Index(['paraphrase_mpnet_base_v2_emb_1', 'paraphrase_mpnet_base_v2_emb_2',
       'paraphrase_mpnet_base_v2_emb_3', 'paraphrase_mpnet_base_v2_emb_4',
       'paraphrase_mpnet_base_v2_emb_5', 'paraphrase_mpnet_base_v2_emb_6',
       'paraphrase_mpnet_base_v2_emb_7', 'paraphrase_mpnet_base_v2_emb_8',
       'paraphrase_mpnet_base_v2_emb_9', 'paraphrase_mpnet_base_v2_emb_10',
       ...
       'all_MiniLM_L12_v2_emb_378', 'all_MiniLM_L12_v2_emb_379',
       'all_MiniLM_L12_v2_emb_380', 'all_MiniLM_L12_v2_emb_381',
       'all_MiniLM_L12_v2_emb_382', 'all_MiniLM_L12_v2_emb_383',
       'all_MiniLM_L12_v2_emb_384', 'tweet_length', 'num_words_no_numpunc',
       'num_puctuations'],
      dtype='object', length=3715)


In [79]:
reg = RandomForestRegressor(n_estimators = 700, 
                            max_depth = 10, 
                            max_features = "sqrt",
                            criterion="squared_error",
                            random_state = seed)

In [80]:
model = reg.fit(X_train_final,  y_train_final)

In [81]:
# traning set loss
y_pred_train = model.predict(X_train_final)
y_pred_train = np.clip(y_pred_train, -1, 1)
print(f"RMSE: {mse(y_train_final, y_pred_train)**0.5}")

RMSE: 0.374033633387401


In [82]:
# Evaluation on Validation Set
y_pred_val = model.predict(X_val_final)
y_pred_val = np.clip(y_pred_val, -1, 1)

print(f"RMSE: {mse(y_val_final, y_pred_val)**0.5}")

RMSE: 0.5773622499802337
